# Cuaderno del científico — Proyecto Final (Deep Learning)

**Tema:** Análisis de sentimiento en reseñas de cine (IMDB 50K)  
**Formato:** Diario de trabajo, decisiones y resultados  

---

## 1) Planteamiento inicial

**Problema.** Clasificar reseñas de películas como positivas o negativas.

**Objetivo cuantitativo.** Superar **70%** en las métricas principales.

**Hipótesis.**
- **H1.** Un “mini-transformer” en Keras con vectorización integrada será suficiente para >70% en IMDB.
- **H2.** Una normalización de texto ligera (lowercase, limpieza de ruido y emojis) mejora estabilidad sin requerir limpieza agresiva.
- **H3.** Con particiones estratificadas y *early stopping*, la validación simple + *bootstrap* en test ofrece evidencia suficiente sin CV exhaustiva.

---

## 2) Diario por etapas

### Semana 1 — Selección de dataset y alcance
Elegí **IMDB 50K Movie Reviews** por tamaño, formato claro y bibliografía abundante.  
Descarté **Sentiment140** y **TweetEval** por ser demasiado grandes y por ende aumentar el tiempo de entrenamiento.  
Definí el enfoque técnico: **“mini-transformer”** con Keras y un **70%** como objetivo mínimo en métricas principales.

### Semana 2 — EDA y preparación
Hice una inspección rápida:  
- Ruido típico (HTML, etiquetas, símbolos) → filtrado básico.  
- Emojis y unicode: baja frecuencia; añadí tiempo en adelante gestión en la función de *standardize* de `TextVectorization`.  
- Reseñas largas y coloquiales; confirmé la necesidad de truncado por `seq_len`.  
Dejé para más adelante: análisis de longitud por clase y vocabulario exhaustivo (no críticos para el MVP).

### Semana 3 — Particionado y protocolo experimental
Apliqué **train / valid / test** con **estratificación**. El **test** quedó totalmente vedado hasta el final para evitar *leakage*.  
Exploré *wrappers* `KerasClassifier` (scikeras) + `StratifiedKFold` con *multi-métricas*, pero **cambié de plan**: la **CV exhaustiva se abandonó** por coste/beneficio; pasé a **validación interna estratificada** durante el ajuste y **bootstrap en test** para estimar incertidumbre.

### Semana 4 — Ingeniería de texto y modelo
Integré `TextVectorization` en el propio grafo del modelo para aceptar texto en crudo en inferencia.  
Ajusté *standardize* para emojis/unicode y fijé `vocab_size` a partir de conteos piloto.  
Arquitectura final: **mini-transformer** con *embedding*, bloque de atención ligera y *feed-forward*; umbral de decisión (`threshold`) configurable desde el *config*.  
**Ajuste operativo de entrenamiento:** fijé **epochs = 3** y **seq_len = 256** para tiempos de entrenamiento más razonables. En coherencia con esto, **retiré los *callbacks*** *EarlyStopping* y *ReduceLROnPlateau* (tenían **patience** 2 y 1, con poca utilidad en tan pocos *epochs*). Se mantuvo el objetivo de rendimiento y la estabilidad del entrenamiento.  
Métricas rastreadas: F1 / ROC-AUC / Accuracy según necesidad; el objetivo **>70%** se cumplió.  
Incidencias: una ruta de cálculo del **OOV** se volvió muy lenta tras variar `seq_len`; se estabilizó al revisar canalización y límites de secuencia. Sucedia principalmente cuando no aparecía el **OUT_OF_RANGE: End of sequence** asociado a `TextVectorization`; se solucionó tras ajustar algunos valores de configuración.

### Semana 5 — Limpieza, configuración y *hardening*
Reorganicé la **configuración** para reflejar el abandono del CV; añadí `threshold` y `prediction_confidence`.  
Ordené el código con **tipados**, **docstrings** y **markdowns**; limpié el `requirements.txt` de los paquetes que no se terminaron usando.  
Unifiqué los textos de codigo al **inglés** y los textos de usuario al **español**.  

### Semana 6 — Producto mínimo y despliegue
Entrené el **modelo final** con todos los datos de entrenamiento y lo guardé en formato `SavedModel`;  
Exposé el modelo vía **API REST** y la **dockericé**; limpié el codigo de comentarios y añadí la función de `prediction_confidence` a la API.
Finalmente pulí la documentación del proyecto, los README y el diario de trabajo.

---

## 3) Resultados y evidencias

- **Objetivo cuantitativo:** superado el **70%** en métricas principales.  
- **Comportamiento:** normalización ligera + `TextVectorization` en el grafo simplifica el *serving* y mantiene rendimiento.  
- **Eficiencia:** con **epochs = 3** y **seq_len = 256** se obtuvieron **tiempos de entrenamiento más razonables** sin degradar el objetivo mínimo.  
- **Robustez:** *bootstrap* sobre el conjunto de test para bandas de incertidumbre con coste de cómputo controlado.

---

## 4) Decisiones clave (y por qué)

1. **IMDB 50K sobre datasets de Twitter.** Menos ruido de plataforma y menor volumen de datos para un mejor tiempo de entrenamiento.  
2. **Sin CV exhaustiva.** Coste alto y ganancia marginal baja; sustitución por validación interna estratificada y bootstrap en test.
3. **Normalización ligera.** Suficiente para el objetivo sin complicar el pipeline.